In [41]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from keras.utils import image_dataset_from_directory
from sklearn.ensemble import RandomForestClassifier

In [2]:
test_df="/kaggle/input/ovarian-cancer-classification-dataset/Test_Images"
train_df="/kaggle/input/ovarian-cancer-classification-dataset/Train_Images"


In [3]:
c=cv2.imread("/kaggle/input/ovarian-cancer-classification-dataset/Train_Images/EC/10029.png")
c.shape

(224, 224, 3)

In [4]:
# train_data=[]
# for cur,dir,files in os.walk(train_df):
#     for file in files:
#         data=os.path.join(cur,file)
#         data_cv=cv2.imread(data)
# #         \d
# #         data=cv2.imread(file)
#         if data is not None:

#             train_data.append(data_cv)
# for i in range(10):
#     plt.imshow(train_data[i])
#     plt.show()

In [5]:
training_data=image_dataset_from_directory(
train_df,
seed=123,
# validation_split=0.2,
batch_size=32,
shuffle=True,
image_size=(224,224))
# subset="training")
validation_data=image_dataset_from_directory(
test_df,
seed=123,
# validation_split=0.2,
shuffle=True,

batch_size=32,
image_size=(224,224))
# subset="validation")

Found 31203 files belonging to 5 classes.
Found 3082 files belonging to 5 classes.


# # Machine Learning

In [6]:
from sklearn.linear_model import LinearRegression,RidgeClassifier,RidgeClassifierCV,LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier,StackingClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
# import Bokeh
# import Altair
# import Plotly

In [7]:
# random_classif=RandomForestClassifier()
# random_classif.fit(training_data)
# random_classif.score(validation_data)

In [8]:
# X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [9]:
# base_estimator=[
#     ("rid_class",RidgeClassifier()),
#     ("ridgeClasf_cv",RidgeClassifierCV()),
#     ("mlp_clasf",MLPClassifier()),
#     ("gaus_nb",GaussianNB()),
#     ("k_neigh_clasf",KNeighborsClassifier()),
#     ("radius_clasf",RadiusNeighborsClassifier()),
#     ("catboost",CatBoostClassifier()),
#     ("xbgboost",XGBClassifier()),
#     ("decision_tree",DecisionTreeClassifier()),
#     ("random_forest",RandomForestClassifier())
    
    
# ]
# staking_algo=StackingClassifier(estimators=base_estimator,final_estimator=SVC())
# staking_algo.fit(X_train,Y_train)
# staking_algo.score(X_train,Y_train)
# stcking_algo.score(X_test,Y_test)

In [26]:
def add_extra_dim(image,label):
    label=tf.expand_dims(label,axis=-1)
    return image, label
training_label_data=training_data.map(add_extra_dim)
val_label_data=validation_data.map(add_extra_dim)

In [27]:
for image,label in val_label_data.take(3):
    print(label.shape)

(32, 1)
(32, 1)
(32, 1)


# # Deep Learning

In [10]:
import keras
import tensorflow as tf
from keras.layers import Dense,Input,Flatten,Embedding,Conv2D,MaxPooling2D,Dropout,BatchNormalization,UpSampling2D
from keras.models import Sequential,Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [11]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [35]:
inputs=Input(shape=(224,224,3))
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
M=MaxPooling2D(2,2)(C)
B=BatchNormalization()(M)
C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
M=MaxPooling2D(2,2)(C)
F=Flatten()(M)

# C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
# C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
# M=UpSampling2D(size=(2,2))(C)
# B=BatchNormalization()(M)
# C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
# C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
# M=UpSampling2D(size=(2,2))(C)
D=Dense(32,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(F)
D=Dense(32,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
# outputs=Conv2D(5,(1,1),activation="softmax",padding="same")(M)
outputs=Dense(5,activation="softmax")(D)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [36]:
model.fit(training_label_data,epochs=50,validation_data=val_label_data,callbacks=[early_stopping])

Epoch 1/50
  1/976 ━━━━━━━━━━━━━━━━━━━━ 6:26:07 24s/step - accuracy: 0.1250 - loss: 7.5635

I0000 00:00:1727236788.823592     526 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1727236788.844090     526 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


976/976 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.3710 - loss: 7.6334

W0000 00:00:1727236883.302220     525 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


976/976 ━━━━━━━━━━━━━━━━━━━━ 127s 106ms/step - accuracy: 0.3710 - loss: 7.6305 - val_accuracy: 0.3838 - val_loss: 2.8951
Epoch 2/50
976/976 ━━━━━━━━━━━━━━━━━━━━ 94s 96ms/step - accuracy: 0.4175 - loss: 2.4927 - val_accuracy: 0.3855 - val_loss: 2.0169
Epoch 3/50
976/976 ━━━━━━━━━━━━━━━━━━━━ 95s 97ms/step - accuracy: 0.4377 - loss: 1.7856 - val_accuracy: 0.3297 - val_loss: 1.7093
Epoch 4/50
976/976 ━━━━━━━━━━━━━━━━━━━━ 95s 97ms/step - accuracy: 0.4429 - loss: 1.5521 - val_accuracy: 0.3008 - val_loss: 1.7109
Epoch 5/50
976/976 ━━━━━━━━━━━━━━━━━━━━ 94s 96ms/step - accuracy: 0.4509 - loss: 1.4794 - val_accuracy: 0.3504 - val_loss: 2.0114
Epoch 6/50
976/976 ━━━━━━━━━━━━━━━━━━━━ 94s 95ms/step - accuracy: 0.4553 - loss: 1.4279 - val_accuracy: 0.2554 - val_loss: 1.6862
Epoch 7/50
976/976 ━━━━━━━━━━━━━━━━━━━━ 93s 95ms/step - accuracy: 0.4610 - loss: 1.3638 - val_accuracy: 0.4452 - val_loss: 1.3981
Epoch 8/50
976/976 ━━━━━━━━━━━━━━━━━━━━ 93s 95ms/step - accuracy: 0.4750 - loss: 1.3442 - val_accur

In [38]:
loss,accuracy=model.evaluate(val_label_data)

97/97 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.3537 - loss: 1.5452


In [37]:
model.summary()

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_81 (Conv2D)              │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_82 (Conv2D)              │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_83 (Conv2D)              │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_84 (Conv2D)              │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 200704)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │     6,422,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,468,305 (74.27 MB)

 Trainable params: 6,489,413 (24.76 MB)

 Non-trainable params: 64 (256.00 B)

 Optimizer params: 12,978,828 (49.51 MB)

In [ ]:
import numpy as np
train_features = []
train_labels = []

for images, labels in training_label_data:
    features = model.predict(images)
    train_features.append(features)
    train_labels.append(labels)
train_features = np.concatenate(train_features)
train_labels = np.concatenate(train_labels)


In [44]:
import numpy as np
val_features = []
val_labels = []

for images, labels in val_label_data:
    features = model.predict(images)
    val_features.append(features)
    val_labels.append(labels)

val_features = np.concatenate(val_features)
val_labels = np.concatenate(val_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━

In [ ]:
# X_train,X_test,Y_train,Y_test=train_test_split(train_feature,)

In [43]:
random_classif=RandomForestClassifier()
random_classif.fit(train_features,train_labels)

/tmp/ipykernel_470/430169262.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_classif.fit(train_features,train_labels)


RandomForestClassifier()

In [45]:
random_classif.score(val_features,val_labels)

0.3887086307592472